IMPORTING THE LIBRARIES

In [1]:
import open3d as o3d
import numpy as np
import importlib
import matplotlib.pyplot as plt
import h5py
import functions.pc_matching
import functions.mat_to_py
import functions.viz
import functions.pc_registration
importlib.reload(functions.pc_matching)
importlib.reload(functions.mat_to_py)
importlib.reload(functions.viz)
importlib.reload(functions.pc_registration)
from functions.pc_matching import PCMatch
from functions.mat_to_py import PCNIData
from functions.viz import pc_viz, video_cv, pc2video
from functions.pc_registration import PointReg
import cv2 as cv
import argparse
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Data Collection

In [2]:
datapoints = PCNIData('C:\\Users\\dasgu\\Documents\\GithubPercep\\sensordata\\Sept_Recording\\Sep28-2022_CAM-LCR_LIDAR_applanix\\14-50-47',eg=2)
images = datapoints('image')

loading  C:\Users\dasgu\Documents\GithubPercep\sensordata\Sept_Recording\Sep28-2022_CAM-LCR_LIDAR_applanix\14-50-47\imgdata\2022-06-16-13-01-40_2_img.mat


Change of Code


In [3]:
import functions.imageprediction
importlib.reload(functions.imageprediction)
from functions.imageprediction import imagepred
img_pred = imagepred() 
# img = images[150]
# img = cv.imread('image_1_1000.png')
# cv.imshow('PC',  img)
# cv.waitKey(-1)
img_pred(images,150,160,3)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [4]:
# img = cv.imread(img[200])
cv.imshow('PC',  img)
cv.waitKey(-1)

# Give the configuration and weight files for the model and load the network.
net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

In [158]:
imag=img.copy()
cv.rectangle(imag, (10,10), (100,100),'blue', 2)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Scalar value for argument 'color' is not numeric
>  - Scalar value for argument 'color' is not numeric
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


In [5]:
img = cv.imread('image_1_1000.png')

In [6]:

def get_output_layers(net):   
    layer_names = net.getLayerNames()   
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv.putText(img, label, (x-10,y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    

    
Width = img.shape[1]
Height = img.shape[0]
scale = 0.00392

classes = None

with open('yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
net = cv.dnn.readNetFromDarknet('yolov4-obj.cfg', 'yolov4-obj_final.weights')
blob = cv.dnn.blobFromImage(img, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))

class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4



In [8]:

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.7:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])
indices = cv.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
print(len(indices))
img = np.ascontiguousarray(img, dtype=np.uint8)
for i in indices:
    # prin
    box = boxes[i]
    print(box)
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(img, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

cv.imshow("object detection", img)
cv.waitKey()  
cv.imwrite("object-detection.jpg", img)
cv.destroyAllWindows()

1
[126.0, 185.0, 132, 80]
